$\large\textbf{Lab 7.} $


$\large\textbf{Exercise 1 : The need for regularization}$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
import math

In [ ]:
digits = load_digits()
#check the shape of digits data
print(digits.data.shape)
#check the shape of digits target
print(digits.target.shape)
#let us use the linear regression used in the previous lab
N = digits.data.shape[0] #Number of data points
d = digits.data.shape[1] #Dimension of data points
A = digits.data
#In the following code, we create a Nx1 vector of target labels
y = 1.0*np.ones([A.shape[0],1])
for i in range(digits.target.shape[0]):
 y[i] = digits.target[i]


(1797, 64)
(1797,)


In [ ]:
 def evalf(x,lam):  
  assert type(x) is np.ndarray   
  fval = np.linalg.norm(np.matmul(A,x) - y)
  fval = 0.5 * (fval)**2+(lam/2)*np.matmul(x.T,x)
  return (fval)

In [ ]:
def evalg(x,lam):
  assert type(x) is np.ndarray 
  grad=(lam*x+np.matmul(A.T, np.matmul(A, x) - y))
  return grad

In [ ]:
def evalh(x,lam):
  assert type(x) is np.ndarray  
  return np.matmul(A.T,A)+lam*np.identity(d)

In [ ]:
def compute_B_k(s, y, B_k):
  I=np.identity(64)
  assert type(s) is np.ndarray 
  assert type(y) is np.ndarray 
  assert type(B_k) is np.ndarray
  mu = 1 / np.dot(np.squeeze(np.asarray(y)), np.squeeze(np.asarray(s)))
  a1 = np.outer(np.multiply(mu, s), np.transpose(y))
  a2 = np.outer(np.multiply(mu, y), np.transpose(s))
  a = np.subtract(I, a1)
  b = np.subtract(I, a2)
  c = np.outer(np.multiply(mu, s), np.transpose(s))
  d = np.matmul(np.matmul(a, B_k), b)
  z = np.add(d, c)
  return z

In [ ]:
#line search type 
CONSTANT_STEP_LENGTH = 3
BACKTRACKING_LINE_SEARCH = 2
EXACT_LINE_SEARCH = 1

In [ ]:
def compute_D_k(x):
  assert type(x) is np.ndarray
  mat = evalh(x)
  return np.linalg.inv(mat)

In [ ]:
def compute_steplength_backtracking_scaled_direction(x, lam,gradf, alpha_start, rho, gamma, B_k): 
  assert type(x) is np.ndarray
  assert type(gradf) is np.ndarray
  assert type(alpha_start) is float and alpha_start>=0. 
  assert type(rho) is float and rho>=0.
  assert type(gamma) is float and gamma>=0. 
  alpha = alpha_start
  p = - gradf
  D_k = B_k

      
  while evalf(x + alpha*np.matmul(D_k,p),lam) > evalf(x,lam) + gamma * alpha* (np.matmul(np.matrix.transpose(gradf), np.matmul(D_k,p)) ):
    alpha = alpha * rho
  return alpha 

In [ ]:
def find_minimizer_Newtonmethod(start_x,lam, tol, line_search_type, *args):
  assert type(start_x) is np.ndarray  
  assert type(tol) is float and tol>=0 
  
  x = start_x
  g_x = evalg(x,lam)
  h_x = evalh(x,lam)

  if line_search_type == BACKTRACKING_LINE_SEARCH:
    if args is None:
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive any args. Please check!'
      raise ValueError(err_msg)
    elif len(args)<3 :
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive three args. Please check!'
      raise ValueError(err_msg)
    else:
      alpha_start = float(args[0])
      rho = float(args[1])
      gamma = float(args[2])
  k = 0
  x_array_newton = []
  f_array_newton = []
  while (np.linalg.norm(g_x) > tol): 
    D_k = np.linalg.inv(evalh(x,lam))
    if line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking_scaled_direction(x,lam, g_x, alpha_start, rho, gamma, D_k)
    elif line_search_type == CONSTANT_STEP_LENGTH: 
      step_length = 1.0
    else:
      raise ValueError('Line search type unknown. Please check!') 
    x = np.subtract(x, np.multiply(step_length,np.matmul(D_k, g_x))) 
    k = k + 1
    g_x = evalg(x,lam) 
  return x,  k

In [ ]:
def find_minimizer_BFGS_scaling(start_x, lam,tol, line_search_type, *args):
  assert type(start_x) is np.ndarray 
  assert type(tol) is float and tol>=0 
  
  x = start_x
  g_x = evalg(x,lam)

  if line_search_type == BACKTRACKING_LINE_SEARCH:
    if args is None:
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive any args. Please check!'
      raise ValueError(err_msg)
    elif len(args)<3 :
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive three args. Please check!'
      raise ValueError(err_msg)
    else:
      alpha_start = float(args[0])
      rho = float(args[1])
      gamma = float(args[2])

  k = 0
  B_k = np.identity(d)
  x_old = x
  x_array_bfgs = []
  f_array_bfgs = []
  while (np.linalg.norm(g_x) > tol): 
    if line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking_scaled_direction(x_old,lam, g_x, alpha_start, rho, gamma, B_k)
    elif line_search_type == CONSTANT_STEP_LENGTH: 
      step_length = 1.
    else:
      raise ValueError('Line search type unknown. Please check!')
    x_new = np.subtract(x_old, np.multiply(step_length, np.matmul(B_k,g_x))) 
    g_x = evalg(x_new,lam)
    s = np.subtract(x_new, x_old)
    y = np.subtract(evalg(x_new,lam), evalg(x_old,lam)) 
    B_k = compute_B_k(s, y, B_k)
    x_old = x_new
    k = k + 1
  return x_new, k

$\large\textbf{Que 1: [R]}$ Now use your Newton method to solve problem (1), which is the direct OLSLR. Use the starting point x = 0. Indicate the difficulties you encounter. Check if you face similar difficulties when you use Newton method to solve problem (2), the regularized OLSLR with
λ = 0.1 and starting point x = 0. Explain the reason for your observation. Report the values
of x
∗
f
and x
∗
fλ
.


$\large\textbf{Ans: }$

$\large\textbf{Que 1}$ (1)

In [ ]:
alpha = 0.9
rho = 0.5
gamma = 0.5
my_tol= 1e-3
lam=0
k1=[]
my_start_x =np.zeros(d)
x, k = find_minimizer_Newtonmethod(my_start_x.reshape(d,1),lam, my_tol, BACKTRACKING_LINE_SEARCH, alpha, rho, gamma)
print('for starting with',my_start_x,':\nwe have minimizer:',x ,'\nno. of iterations:',k)

LinAlgError: ignored

$\large\textbf{Que 1}$ (2)

In [ ]:
alpha = 0.9
rho = 0.5
gamma = 0.5
my_tol= 1e-3
lam=0.1
k1=[]
my_start_x =np.zeros(d)
x, k = find_minimizer_Newtonmethod(my_start_x.reshape(d,1),lam, my_tol, BACKTRACKING_LINE_SEARCH, alpha, rho, gamma)
print('for starting with',my_start_x,':\nwe have minimizer:',x ,'\nno. of iterations:',k)


for starting with [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] :
we have minimizer: [[ 0.00000000e+00]
 [ 9.72176393e-02]
 [-4.25221013e-03]
 [-7.65725749e-03]
 [ 7.49359297e-02]
 [ 1.13924666e-02]
 [-2.68134810e-02]
 [-8.48370171e-03]
 [ 9.91208545e-01]
 [-2.87397984e-02]
 [ 1.18690196e-01]
 [ 6.61518399e-02]
 [-5.57615716e-02]
 [-6.96340237e-02]
 [ 9.62813519e-02]
 [ 2.56470858e-01]
 [-7.28979627e-01]
 [ 2.42825856e-02]
 [ 7.72526070e-02]
 [-2.33770172e-02]
 [-5.63320407e-02]
 [ 5.71246069e-02]
 [-4.84767009e-02]
 [-2.70744170e-01]
 [-8.60889237e-01]
 [-1.49941949e-01]
 [ 5.64334649e-02]
 [ 8.96806467e-02]
 [ 8.39114973e-02]
 [ 9.85243348e-02]
 [ 1.64759992e-03]
 [-2.82145749e+00]
 [ 0.00000000e+00]
 [-1.54275472e-01]
 [-9.36618641e-03]
 [ 1.39528972e-01]
 [-3.69438111e-02]
 [ 5.46098301e-02]
 [-9.13188784e-03]
 [ 0.00000000e+00]
 [ 1.

ANS: 

(1) By direct OLSLR,Here we get the singular matrix,hence we can not find the inverse of the matrix.and we also cannot calculate the further calculations.

(2) The regularized OLSLR with λ = 0.1 there are 10 iterations and we have minimizer in above code.

$\large\textbf{2. [R]}$ Use BFGS method with starting point x = 0, to solve problem (1) and describe if you
observe any difficulty. Check if solving the regularized problem (2) helps (use λ = 0.1 and
starting point x = 0). Explain your observations. Report the values of x∗f
and x∗fλ
.

$\large\textbf{2.}$ (1)

In [ ]:
alpha = 0.9
rho = 0.5
gamma = 0.5
my_tol= 1e-5
lam=0
k2=[]

my_start_x =np.zeros(d)

x, k  = find_minimizer_BFGS_scaling(my_start_x.reshape((d,1)),lam, my_tol, BACKTRACKING_LINE_SEARCH, alpha, rho, gamma)
print('for starting with',my_start_x,':\nwe have minimizer:',x ,'\nno. of iterations:',k)

for starting with [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] :
we have minimizer: [[ 0.00000000e+00]
 [ 9.69033568e-02]
 [-4.32277232e-03]
 [-7.76028319e-03]
 [ 7.49594380e-02]
 [ 1.13947198e-02]
 [-2.71328245e-02]
 [-7.33176333e-03]
 [ 9.98337968e-01]
 [-2.88095538e-02]
 [ 1.18688288e-01]
 [ 6.60916265e-02]
 [-5.57069862e-02]
 [-6.97063705e-02]
 [ 9.65876439e-02]
 [ 2.55182251e-01]
 [-7.29828608e-01]
 [ 2.42709916e-02]
 [ 7.73249597e-02]
 [-2.33000278e-02]
 [-5.64086144e-02]
 [ 5.72426822e-02]
 [-4.88717684e-02]
 [-2.62467763e-01]
 [-9.06562829e-01]
 [-1.49767791e-01]
 [ 5.64019538e-02]
 [ 8.96663590e-02]
 [ 8.39318159e-02]
 [ 9.85411936e-02]
 [ 1.69317614e-03]
 [-2.96805758e+00]
 [ 0.00000000e+00]
 [-1.54362338e-01]
 [-9.32361205e-03]
 [ 1.39497628e-01]
 [-3.69234835e-02]
 [ 5.46111776e-02]
 [-9.20505070e-03]
 [ 0.00000000e+00]
 [ 1.

$\large\textbf{2.}$ (2)

In [ ]:
alpha = 0.9
rho = 0.5
gamma = 0.5
my_tol= 1e-5
lam=0.1
k2=[]

my_start_x =np.zeros(d)

x, k  = find_minimizer_BFGS_scaling(my_start_x.reshape((d,1)),lam, my_tol, BACKTRACKING_LINE_SEARCH, alpha, rho, gamma)
print('for starting with',my_start_x,':\nwe have minimizer:',x ,'\nno. of iterations:',k)  

for starting with [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] :
we have minimizer: [[ 0.00000000e+00]
 [ 9.72176393e-02]
 [-4.25221011e-03]
 [-7.65725749e-03]
 [ 7.49359298e-02]
 [ 1.13924666e-02]
 [-2.68134811e-02]
 [-8.48370171e-03]
 [ 9.91208545e-01]
 [-2.87397984e-02]
 [ 1.18690196e-01]
 [ 6.61518400e-02]
 [-5.57615717e-02]
 [-6.96340237e-02]
 [ 9.62813519e-02]
 [ 2.56470858e-01]
 [-7.28979628e-01]
 [ 2.42825856e-02]
 [ 7.72526071e-02]
 [-2.33770172e-02]
 [-5.63320407e-02]
 [ 5.71246069e-02]
 [-4.84767009e-02]
 [-2.70744170e-01]
 [-8.60889235e-01]
 [-1.49941949e-01]
 [ 5.64334649e-02]
 [ 8.96806467e-02]
 [ 8.39114973e-02]
 [ 9.85243348e-02]
 [ 1.64759992e-03]
 [-2.82145749e+00]
 [ 0.00000000e+00]
 [-1.54275472e-01]
 [-9.36618641e-03]
 [ 1.39528972e-01]
 [-3.69438111e-02]
 [ 5.46098301e-02]
 [-9.13188785e-03]
 [ 0.00000000e+00]
 [ 1.

ANS:

for BFGS method there are 72 iterations in both of the parts and the function values are not much different because the lamda value are not much vary .
